Detected 12 layers in the custom model
MATCH (Explicit) transformer.transformer.layers.0.self_attn.out_proj.weight transformer.h.0.attn.c_proj.weight
MATCH (Explicit) transformer.transformer.layers.0.self_attn.out_proj.bias transformer.h.0.attn.c_proj.bias
MATCH (Explicit) transformer.transformer.layers.1.self_attn.out_proj.weight transformer.h.1.attn.c_proj.weight
MATCH (Explicit) transformer.transformer.layers.1.self_attn.out_proj.bias transformer.h.1.attn.c_proj.bias
MATCH (Explicit) transformer.transformer.layers.2.self_attn.out_proj.weight transformer.h.2.attn.c_proj.weight
MATCH (Explicit) transformer.transformer.layers.2.self_attn.out_proj.bias transformer.h.2.attn.c_proj.bias
MATCH (Explicit) transformer.transformer.layers.3.self_attn.out_proj.weight transformer.h.3.attn.c_proj.weight
MATCH (Explicit) transformer.transformer.layers.3.self_attn.out_proj.bias transformer.h.3.attn.c_proj.bias
MATCH (Explicit) transformer.transformer.layers.4.self_attn.out_proj.weight transformer.h

In [1]:


import torch
input = torch.load("test.pt")

from transformers import GPT2LMHeadModel
from one_step_thoughtsformer import ThoughtsFormer
# t = GPT2Tokenizer.from_pretrained("gpt2")
pret = GPT2LMHeadModel.from_pretrained("gpt2")
custom_model = ThoughtsFormer(num_layers=12,vocab_size=50257, d_embed = 768, feed_forward_dim=768 * 4, n_head=12, max_context_length=1024, thought_length=0, dropout=0.15, sinusoidal_position_encoding=False)
pdict = pret.state_dict()
cdict = custom_model.state_dict()
# sentence = "Once upon a time"
pdict = pret.state_dict()
cdict = custom_model.state_dict()

explicit_map = {
    'token_embedding.weight' : 'transformer.wte.weight',
    'policy_feedforward.weight' : 'lm_head.weight',
    'transformer.transformer.norm.weight' : 'transformer.ln_f.weight',
    'transformer.transformer.norm.bias' : 'transformer.ln_f.bias',
    'transformer.dual_positional_encoding.learned_positional_encoding.weight' : 'transformer.wpe.weight'
}

param_counter = 0

for k, v in explicit_map.items():
    param_counter+= 1
    assert cdict[k].shape == pdict[v].shape
    cdict[k].copy_(pdict[v])
    
for i in range(12):
    layer_explicit_map = {
        f"transformer.transformer.layers.{i}.linear1.weight" : f"transformer.h.{i}.mlp.c_fc.weight" ,
        f"transformer.transformer.layers.{i}.linear1.bias" : f"transformer.h.{i}.mlp.c_fc.bias" ,
        f"transformer.transformer.layers.{i}.linear2.weight" : f"transformer.h.{i}.mlp.c_proj.weight", 
        f"transformer.transformer.layers.{i}.linear2.bias" : f"transformer.h.{i}.mlp.c_proj.bias" ,
        f"transformer.transformer.layers.{i}.norm1.weight" : f"transformer.h.{i}.ln_1.weight", 
        f"transformer.transformer.layers.{i}.norm1.bias" : f"transformer.h.{i}.ln_1.bias", 
        f"transformer.transformer.layers.{i}.norm2.weight" : f"transformer.h.{i}.ln_2.weight", 
        f"transformer.transformer.layers.{i}.norm2.bias" : f"transformer.h.{i}.ln_2.bias", 
        f"transformer.transformer.layers.{i}.self_attn.out_proj.weight" : f"transformer.h.{i}.attn.c_proj.weight", 
        f"transformer.transformer.layers.{i}.self_attn.out_proj.bias" : f"transformer.h.{i}.attn.c_proj.bias",
        f"transformer.transformer.layers.{i}.self_attn.in_proj_weight":  f"transformer.h.{i}.attn.c_attn.weight",
        f"transformer.transformer.layers.{i}.self_attn.in_proj_bias" : f"transformer.h.{i}.attn.c_attn.bias"
    }
    
    transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
    for k, v in layer_explicit_map.items():
        
        if any(v.endswith(w) for w in transposed):
            assert pdict[v].shape[::-1] == cdict[k].shape
            with torch.no_grad():
                cdict[k].copy_(pdict[v].t())
        else:
            # print(cdict[k].shape, pdict[v].shape, k, v)
            with torch.no_grad():
                assert cdict[k].shape == pdict[v].shape
                cdict[k].copy_(pdict[v])
    
custom_model.load_state_dict(cdict)
# cdict['token_embedding.weight'].shape
# cdict['transformer.transformer.layers.0.linear2.weight'].shape

sentence = "once upon a time"


from bpe import BPETokenizer
tokenizer = BPETokenizer()

torch.manual_seed(42)
with torch.no_grad():
    tokens = torch.tensor(tokenizer(sentence)).view(1,-1)
    custom_model.eval()
    predictions = custom_model.forward_ppo_with_tokens(tokens, torch.zeros_like(tokens),0)[0]
  
    print(custom_model.transformer.transformer.layers[0](input))
    
    
    print(custom_model.transformer.transformer.layers[0](input))

x = predictions.argmax(dim=-1)
tokenizer.decode(x[0])
print("\n\nDesired Output")
print(pret(tokens,output_hidden_states=True).hidden_states[1])



NameError: name 'torch' is not defined

In [ ]:
# Desired Output
# tensor([[[ 0.3191,  0.6466,  1.1327,  ..., -0.0743,  0.7379, -0.3372],
#          [-0.3657, -0.3845, -0.2498,  ..., -0.0071,  0.2876,  0.2585],
#          [-1.4182,  0.1991, -0.4063,  ...,  0.0237,  0.1473, -0.4916],
#          [ 0.2380, -0.8441, -0.0074,  ...,  0.6175, -0.6014,  0.4939]]],
#        grad_fn=<AddBackward0>)

In [10]:
from transformers.models.gpt2.modeling_gpt2 import GPT2Block
desired_layer: GPT2Block = pret.transformer.h[0]
print(input.shape)
desired_layer.forward(input)

torch.Size([1, 1024, 768])


(tensor([[[ 0.3191,  0.6466,  1.1327,  ..., -0.0743,  0.7379, -0.3372],
          [-0.3657, -0.3845, -0.2498,  ..., -0.0071,  0.2876,  0.2585],
          [-1.4182,  0.1991, -0.4063,  ...,  0.0237,  0.1473, -0.4916],
          ...,
          [-0.0923,  0.2690, -0.1589,  ..., -1.0229, -0.9910, -0.2968],
          [-0.0926,  0.3155, -0.1734,  ..., -0.9878, -0.9938, -0.2910],
          [-0.0140, -0.0124, -0.1848,  ..., -0.5759, -0.4561, -0.2137]]],
        grad_fn=<AddBackward0>),
 None)

In [4]:
import torch
import torch.nn as nn
encoder_layer: nn.TransformerEncoderLayer = custom_model.transformer.transformer.layers[0]
mask = torch.zeros_like(input[:,:,0]).bool()
mask[0,-1] = True
mask[0,-2] = True
mask[0,-3] = True

# import torch.nn as nn
# encoder_layer: nn.TransformerEncoderLayer = custom_model.transformer.transformer.layers[0]
# mask = torch.zeros_like(input[:,:,0]).bool()
# mask[0,-1] = True
# mask[0,-2] = True
# mask[0,-3] = True

# after:  tensor([[[ 0.0275,  0.0141, -0.0031,  ...,  0.5349,  0.2867,  0.1896],
#          [ 0.0340,  0.1176,  0.0336,  ...,  1.1702,  0.1436,  0.4704],
#          [ 0.0355,  0.1407,  0.0450,  ...,  1.1279,  0.0273,  0.4527],
#          ...,
#          [-0.0236,  0.0494,  0.1736,  ..., -0.4440, -1.1006, -0.5862],
#          [-0.0308,  0.0440,  0.1965,  ..., -0.6584, -1.1000, -0.5023],
#          [ 0.0411,  0.1527, -0.0938,  ...,  0.5289,  0.0380,  0.5038]]]

encoder_layer.forward(input, src_mask=mask, is_causal=True)


before:  tensor([[[-0.0665, -0.0487,  0.2646,  ...,  0.2566, -0.1802,  0.0786],
         [-0.0087, -0.0530, -0.0870,  ..., -0.0863, -0.0397,  0.0746],
         [-0.0464, -0.0791,  0.1016,  ...,  0.0623,  0.0928, -0.0598],
         ...,
         [-0.0018,  0.0016, -0.0551,  ...,  0.0136, -0.0072,  0.0038],
         [ 0.0032,  0.0016, -0.0489,  ...,  0.0207, -0.0118, -0.0006],
         [ 0.0003,  0.0030, -0.0017,  ..., -0.0047, -0.0024, -0.0058]]])
here!
after:  tensor([[[ 0.0416,  0.0467,  0.1206,  ...,  0.3639, -0.0996, -0.0363],
         [ 0.0356,  0.0785,  0.1018,  ...,  0.8601,  0.0294,  0.2885],
         [ 0.0628,  0.1810,  0.1949,  ...,  1.0987, -0.8252,  0.0203],
         ...,
         [-0.0205,  0.0474,  0.1777,  ..., -0.4271, -1.1346, -0.6030],
         [-0.0198,  0.0479,  0.1740,  ..., -0.4534, -1.1293, -0.5940],
         [ 0.0410,  0.1495, -0.0950,  ...,  0.5224,  0.0520,  0.5131]]],
       grad_fn=<NativeLayerNormBackward0>)


tensor([[[ 0.0416,  0.0467,  0.1206,  ...,  0.3639, -0.0996, -0.0363],
         [ 0.0356,  0.0785,  0.1018,  ...,  0.8601,  0.0294,  0.2885],
         [ 0.0628,  0.1810,  0.1949,  ...,  1.0987, -0.8252,  0.0203],
         ...,
         [-0.0205,  0.0474,  0.1777,  ..., -0.4271, -1.1346, -0.6030],
         [-0.0198,  0.0479,  0.1740,  ..., -0.4534, -1.1293, -0.5940],
         [ 0.0410,  0.1495, -0.0950,  ...,  0.5224,  0.0520,  0.5131]]],
       grad_fn=<WhereBackward0>)

In [3]:
input = torch.load("test.pt")
print(input)

tensor([[[-0.0665, -0.0487,  0.2646,  ...,  0.2566, -0.1802,  0.0786],
         [-0.0087, -0.0530, -0.0870,  ..., -0.0863, -0.0397,  0.0746],
         [-0.0464, -0.0791,  0.1016,  ...,  0.0623,  0.0928, -0.0598],
         ...,
         [-0.0018,  0.0016, -0.0551,  ...,  0.0136, -0.0072,  0.0038],
         [ 0.0032,  0.0016, -0.0489,  ...,  0.0207, -0.0118, -0.0006],
         [ 0.0003,  0.0030, -0.0017,  ..., -0.0047, -0.0024, -0.0058]]],
       requires_grad=True)


C:\Users\bicke\AppData\Local\Temp\ipykernel_38300\3038415613.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  input = torch.load("test.pt")


In [11]:
print("hi")
print("hi")
print("hi")
print("hi")
print("hi")

hi
hi
hi
hi
hi


In [38]:
import torch
import torch.nn.functional as F

def compare_transformer_layer_side_by_side(input_tensor, layer_idx, cdict, pdict, padding_mask=None, causal_mask=None):
    """
    Compare transformer layer computations (QKV, attention, feedforward) side-by-side with masking and normalization.

    Args:
        input_tensor: Tensor of shape [batch_size, seq_len, d_model].
        layer_idx: Index of the transformer layer to compare.
        cdict: Dictionary with weights from the first model.
        pdict: Dictionary with weights from the second model.
        padding_mask: Tensor of shape [batch_size, 1, seq_len] indicating padded tokens (0 = pad, 1 = valid).
        causal_mask: Tensor of shape [seq_len, seq_len] with -inf on invalid future positions for causal masking.

    Returns:
        None. Prints side-by-side comparisons.
    """

    def get_weight(d, name):
        """Retrieve weights from the appropriate dictionary."""
        return d[name]

    # Step 1: Layer Norm 1 (before self-attention)
    c_ln1_weight = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.norm1.weight")
    c_ln1_bias = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.norm1.bias")

    p_ln1_weight = get_weight(pdict, f"transformer.h.{layer_idx}.ln_1.weight")
    p_ln1_bias = get_weight(pdict, f"transformer.h.{layer_idx}.ln_1.bias")

    c_normed_input = F.layer_norm(input_tensor, input_tensor.shape[-1:], c_ln1_weight, c_ln1_bias)
    p_normed_input = F.layer_norm(input_tensor, input_tensor.shape[-1:], p_ln1_weight, p_ln1_bias)

    # Step 2: In-Projection for QKV
    c_in_proj_weight = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.self_attn.in_proj_weight")
    c_in_proj_bias = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.self_attn.in_proj_bias")

    p_in_proj_weight = get_weight(pdict, f"transformer.h.{layer_idx}.attn.c_attn.weight").T
    p_in_proj_bias = get_weight(pdict, f"transformer.h.{layer_idx}.attn.c_attn.bias")

    c_qkv = F.linear(c_normed_input, c_in_proj_weight, c_in_proj_bias)
    p_qkv = F.linear(p_normed_input, p_in_proj_weight, p_in_proj_bias)

    c_q, c_k, c_v = c_qkv.chunk(3, dim=-1)
    p_q, p_k, p_v = p_qkv.chunk(3, dim=-1)

    # Step 3: Apply Attention Masks
    d_k = c_k.size(-1)

    # Compute raw attention scores
    c_attn_scores = torch.matmul(c_q, c_k.transpose(-2, -1)) / d_k**0.5
    p_attn_scores = torch.matmul(p_q, p_k.transpose(-2, -1)) / d_k**0.5

    # Apply padding mask: Broadcast and use -inf for masked positions
    if padding_mask is not None:
        padding_mask = padding_mask.unsqueeze(1)  # [batch_size, 1, seq_len]
        c_attn_scores = c_attn_scores.masked_fill(padding_mask == 0, float('-inf'))
        p_attn_scores = p_attn_scores.masked_fill(padding_mask == 0, float('-inf'))

    # Apply causal mask (only if provided)
    if causal_mask is not None:
        c_attn_scores = c_attn_scores + causal_mask
        p_attn_scores = p_attn_scores + causal_mask

    # Softmax over the last dimension
    c_attn_weights = F.softmax(c_attn_scores, dim=-1)
    p_attn_weights = F.softmax(p_attn_scores, dim=-1)

    # Step 4: Attention Output
    c_attn_output = torch.matmul(c_attn_weights, c_v)
    p_attn_output = torch.matmul(p_attn_weights, p_v)

    # Step 5: Out-Projection
    c_out_proj_weight = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.self_attn.out_proj.weight").T
    c_out_proj_bias = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.self_attn.out_proj.bias")

    p_out_proj_weight = get_weight(pdict, f"transformer.h.{layer_idx}.attn.c_proj.weight")
    p_out_proj_bias = get_weight(pdict, f"transformer.h.{layer_idx}.attn.c_proj.bias")

    c_final_attn_output = F.linear(c_attn_output, c_out_proj_weight, c_out_proj_bias)
    p_final_attn_output = F.linear(p_attn_output, p_out_proj_weight, p_out_proj_bias)

    # Step 6: Layer Norm 2 (before feedforward)
    c_ln2_weight = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.norm2.weight")
    c_ln2_bias = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.norm2.bias")

    p_ln2_weight = get_weight(pdict, f"transformer.h.{layer_idx}.ln_2.weight")
    p_ln2_bias = get_weight(pdict, f"transformer.h.{layer_idx}.ln_2.bias")

    c_normed_ffn_input = F.layer_norm(c_final_attn_output, c_final_attn_output.shape[-1:], c_ln2_weight, c_ln2_bias)
    p_normed_ffn_input = F.layer_norm(p_final_attn_output, p_final_attn_output.shape[-1:], p_ln2_weight, p_ln2_bias)

    # Step 7: Feedforward Layers
    c_ffn_weight1 = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.linear1.weight")
    c_ffn_bias1 = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.linear1.bias")

    p_ffn_weight1 = get_weight(pdict, f"transformer.h.{layer_idx}.mlp.c_fc.weight").T
    p_ffn_bias1 = get_weight(pdict, f"transformer.h.{layer_idx}.mlp.c_fc.bias")

    c_ffn_intermediate = F.gelu(F.linear(c_normed_ffn_input, c_ffn_weight1, c_ffn_bias1))
    p_ffn_intermediate = F.gelu(F.linear(p_normed_ffn_input, p_ffn_weight1, p_ffn_bias1))

    c_ffn_weight2 = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.linear2.weight")
    c_ffn_bias2 = get_weight(cdict, f"transformer.transformer.layers.{layer_idx}.linear2.bias")

    p_ffn_weight2 = get_weight(pdict, f"transformer.h.{layer_idx}.mlp.c_proj.weight").T
    p_ffn_bias2 = get_weight(pdict, f"transformer.h.{layer_idx}.mlp.c_proj.bias")

    c_ffn_output = F.linear(c_ffn_intermediate, c_ffn_weight2, c_ffn_bias2)
    p_ffn_output = F.linear(p_ffn_intermediate, p_ffn_weight2, p_ffn_bias2)
    print(c_ffn_output)
    # Step 8: Print Comparisons
    print("=== Comparison Results ===")
    print(f"Layer {layer_idx}: Key Difference (Mean Abs Error): {torch.abs(c_q - p_q).mean().item()}")
    print(f"Layer {layer_idx}: Value Difference (Mean Abs Error): {torch.abs(c_v - p_v).mean().item()}")
    print(f"Layer {layer_idx}: Attention Output Difference (Mean Abs Error): {torch.abs(c_final_attn_output - p_final_attn_output).mean().item()}")
    print(f"Layer {layer_idx}: Feedforward Output Difference (Mean Abs Error): {torch.abs(c_ffn_output - p_ffn_output).mean().item()}")

# Example Usage
# input_tensor = torch.randn(1, 10, 768)  # Example input
padding_mask = torch.ones_like(input[:,:,0])
causal_mask = torch.triu(torch.full((input.size(1), input.size(1)), float('-inf')), diagonal=1)  # Example causal mask

compare_transformer_layer_side_by_side(input, layer_idx=0, cdict=cdict, pdict=pdict, 
                                       padding_mask=padding_mask, causal_mask=causal_mask)


tensor([[[ 22.5750,  -5.0417,   1.8624,  ...,   1.1743,  -0.1306,  18.6665],
         [ 23.3278,  -6.2623,   0.4830,  ...,   2.9826,   1.1707,  24.2999],
         [ 25.9114,  -5.0044, -15.8205,  ...,   4.4183,   6.1842,  17.8748],
         ...,
         [  3.0046, -10.0213,   8.7635,  ...,   0.5806,   4.0622, -12.4676],
         [  3.1045,  -9.8152,   8.4605,  ...,   0.7042,   4.1607, -12.4692],
         [ -0.3725,  -9.3246,   9.0403,  ...,   3.3782,   3.5558, -12.7108]]],
       grad_fn=<ViewBackward0>)
=== Comparison Results ===
Layer 0: Key Difference (Mean Abs Error): 0.0
Layer 0: Value Difference (Mean Abs Error): 0.0
Layer 0: Attention Output Difference (Mean Abs Error): 0.0
Layer 0: Feedforward Output Difference (Mean Abs Error): 0.0


In [30]:
def get_weight(d, name):
        """Retrieve weights from the appropriate dictionary."""
        return d[name]
print(get_weight(cdict, f"transformer.transformer.layers.{0}.self_attn.out_proj.weight"))
print(get_weight(pdict, f"transformer.h.{0}.attn.c_proj.weight"))

tensor([[ 0.3127, -0.2442, -0.4301,  ...,  0.0202,  0.0171, -0.1011],
        [-0.1874, -0.1978,  0.1544,  ...,  0.1984,  0.0252, -0.0735],
        [ 0.0980, -0.1098,  0.0538,  ..., -0.1482, -0.0184, -0.0024],
        ...,
        [-0.0120, -0.0162,  0.0143,  ...,  0.0235, -0.0008,  0.0063],
        [ 0.0094,  0.0250, -0.0181,  ...,  0.0107,  0.0086,  0.0132],
        [-0.0309,  0.0459, -0.0016,  ..., -0.0150,  0.0050, -0.0171]])
tensor([[ 0.3127, -0.1874,  0.0980,  ..., -0.0120,  0.0094, -0.0309],
        [-0.2442, -0.1978, -0.1098,  ..., -0.0162,  0.0250,  0.0459],
        [-0.4301,  0.1544,  0.0538,  ...,  0.0143, -0.0181, -0.0016],
        ...,
        [ 0.0202,  0.1984, -0.1482,  ...,  0.0235,  0.0107, -0.0150],
        [ 0.0171,  0.0252, -0.0184,  ..., -0.0008,  0.0086,  0.0050],
        [-0.1011, -0.0735, -0.0024,  ...,  0.0063,  0.0132, -0.0171]])


In [123]:
#already loaded in 
custom_model.load_state_dict(cdict)

pdict2 = pret.state_dict()
cdict2 = custom_model.state_dict()
# print(pdict2["transformer.h.0.attn.c_attn.weight"].shape)
print(cdict2["transformer.transformer.layers.0.self_attn.in_proj_weight"].shape)
# if False:
i = 0 
layer_explicit_map = {
        f"transformer.transformer.layers.{i}.linear1.weight" : f"transformer.h.{i}.mlp.c_fc.weight" ,
        f"transformer.transformer.layers.{i}.linear1.bias" : f"transformer.h.{i}.mlp.c_fc.bias" ,
        f"transformer.transformer.layers.{i}.linear2.weight" : f"transformer.h.{i}.mlp.c_proj.weight", 
        f"transformer.transformer.layers.{i}.linear2.bias" : f"transformer.h.{i}.mlp.c_proj.bias" ,
        f"transformer.transformer.layers.{i}.norm1.weight" : f"transformer.h.{i}.ln_1.weight", 
        f"transformer.transformer.layers.{i}.norm1.bias" : f"transformer.h.{i}.ln_1.bias", 
        f"transformer.transformer.layers.{i}.norm2.weight" : f"transformer.h.{i}.ln_2.weight", 
        f"transformer.transformer.layers.{i}.norm2.bias" : f"transformer.h.{i}.ln_2.bias", 
        f"transformer.transformer.layers.{i}.self_attn.out_proj.weight" : f"transformer.h.{i}.attn.c_proj.weight", 
        f"transformer.transformer.layers.{i}.self_attn.out_proj.bias" : f"transformer.h.{i}.attn.c_proj.bias",
        f"transformer.transformer.layers.{i}.self_attn.in_proj_weight":  f"transformer.h.{i}.attn.c_attn.weight",
        f"transformer.transformer.layers.{i}.self_attn.in_proj_bias" : f"transformer.h.{i}.attn.c_attn.bias"
    }

# for k,v in layer_explicit_map.items():
#     print(k, " : ", v)
#     if (cdict2[k].ndim == 2):
#         print(cdict2[k][0:2, 0:2], "\n",pdict2[v][0:2, 0:2])
#     else:
#         print(cdict2[k][0:2], "\n",pdict2[v][0:2])
#     print("----")

from bpe import BPETokenizer

tokenizer = BPETokenizer()

x = tokenizer("Once upon a time")
y = custom_model.forward_ppo_with_tokens(x, torch.zeros_like(x),0)

torch.Size([2304, 768])
tensor([[[ 0.0640,  0.0319,  0.2790,  ...,  0.1174, -0.5077, -0.0804],
         [ 0.0231,  0.0954,  0.1616,  ...,  0.6527,  0.1649,  0.4462],
         [ 0.0209,  0.1981,  0.2516,  ...,  1.7870, -1.1436, -0.2067],
         ...,
         [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
         [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
         [    nan,     nan,     nan,  ...,     nan,     nan,     nan]]],
       grad_fn=<NativeLayerNormBackward0>)
tensor([[[-0.0405,  0.0338,  0.1512,  ..., -0.0528, -0.2312, -0.1364],
         [-0.0473, -0.1592, -0.0459,  ..., -0.0747, -0.0718,  0.0230],
         [-0.0591,  0.0242,  0.0416,  ..., -0.0872, -0.2288,  0.0392],
         ...,
         [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
         [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
         [    nan,     nan,     nan,  ...,     nan,     nan,     nan]]],
       grad_fn=<NativeLayerNormBackward0>)
tensor

In [139]:
out = torch.topk(y[0][:,-1,:], k = 5, dim=-1).indices.flatten()

In [140]:
tokenizer.decode(out)

' exha loophsenal�士ortium'

In [105]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = custom_model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate(prompt, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    for _ in range(max_length):
        with torch.no_grad():
            outputs = model.forward_ppo_with_tokens(input_ids, torch.zeros_like(input_ids),0)[0][:,-1,:]
            
        next_token_id = outputs.argmax(dim=-1).unsqueeze(-1)
        input_ids = torch.cat([input_ids, next_token_id], dim=-1)
        print(tokenizer.decode(input_ids[0], skip_special_tokens=True))
        if next_token_id.item() == tokenizer.eos_token_id:
            break

generate("Once upon a time, in a land far away,")

tensor([[[ 3.3518e-02, -2.6569e-02,  2.7954e-01,  ...,  4.5128e-01,
          -1.1943e+00, -3.4916e-01],
         [ 1.3215e-01,  1.4704e-01, -7.9121e-04,  ...,  1.2922e+00,
          -4.8855e-01,  4.5249e-01],
         [ 5.1359e-03,  1.5167e-01,  3.1902e-01,  ...,  3.9099e-01,
          -1.0086e+00, -5.3846e-01],
         ...,
         [        nan,         nan,         nan,  ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan,  ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan,  ...,         nan,
                  nan,         nan]]], device='cuda:0')
tensor([[[ 0.0192, -0.0702, -0.0269,  ..., -0.0646, -0.4809, -0.1334],
         [-0.0166, -0.1049,  0.3334,  ..., -0.0491, -0.2395, -0.0789],
         [ 0.0947,  0.0375,  0.0085,  ..., -0.1998, -0.3473, -0.0491],
         ...,
         [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
         [    nan,     nan,     nan,  .

In [63]:
cdict.keys()

odict_keys(['transformer.dual_positional_encoding.learned_positional_encoding.weight', 'transformer.dual_positional_encoding.position_in_thought_encoding.weight', 'transformer.transformer.layers.0.self_attn.in_proj_weight', 'transformer.transformer.layers.0.self_attn.in_proj_bias', 'transformer.transformer.layers.0.self_attn.out_proj.weight', 'transformer.transformer.layers.0.self_attn.out_proj.bias', 'transformer.transformer.layers.0.linear1.weight', 'transformer.transformer.layers.0.linear1.bias', 'transformer.transformer.layers.0.linear2.weight', 'transformer.transformer.layers.0.linear2.bias', 'transformer.transformer.layers.0.norm1.weight', 'transformer.transformer.layers.0.norm1.bias', 'transformer.transformer.layers.0.norm2.weight', 'transformer.transformer.layers.0.norm2.bias', 'transformer.transformer.layers.1.self_attn.in_proj_weight', 'transformer.transformer.layers.1.self_attn.in_proj_bias', 'transformer.transformer.layers.1.self_attn.out_proj.weight', 'transformer.transfor

In [52]:
cdict['transformer.wte.weight']

tensor([[-0.1101, -0.0393,  0.0331,  ..., -0.1364,  0.0151,  0.0453],
        [ 0.0403, -0.0486,  0.0462,  ...,  0.0861,  0.0025,  0.0432],
        [-0.1275,  0.0479,  0.1841,  ...,  0.0899, -0.1297, -0.0879],
        ...,
        [-0.0445, -0.0548,  0.0123,  ...,  0.1044,  0.0978, -0.0695],
        [ 0.1860,  0.0167,  0.0461,  ..., -0.0963,  0.0785, -0.0225],
        [ 0.0514, -0.0277,  0.0499,  ...,  0.0070,  0.1552,  0.1207]])

13

In [5]:
predictions = custom_model.forward_ppo_with_tokens(tokens, torch.zeros_like(tokens),0)[0]
x = predictions.argmax(dim=-1)
t.decode(x[0])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [33]:
tokens.bool().dtype == torch.int64

False

Number of attention heads: 12


In [3]:
import torch
from transformers import GPT2Tokenizer

def predict_next_token(model: ThoughtsFormer, tokenizer, sentence, device):
    # Tokenize the input sentence
    input_ids = tokenizer.encode(sentence, return_tensors='pt').to(device)
    
    # Generate attention mask
    attention_mask = torch.zeros_like(input_ids).to(device).bool()
    
    # Get the model's prediction
    with torch.no_grad():
        outputs = model.forward_ppo_with_tokens(input_ids, attention_mask, 0)
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs[0]
    
    # Get the predictions for the last token
    next_token_logits = logits[0, -1, :]
    
    # Get the top 5 candidates
    top_5_tokens = torch.topk(next_token_logits, 5, dim=-1)
    
    return top_5_tokens.indices, top_5_tokens.values

# Set up the model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = ThoughtsFormer(vocab_size=50257, d_embed=768, feed_forward_dim=768 * 4, n_head=12, max_context_length=1024, thought_length=0, sinusoidal_position_encoding=False).to(device)

# Load your trained weights
model = custom_model.to(device)
model.eval()

# Test sentence
test_sentence = "The quick brown fox jumps over the lazy"

# Predict next token
top_indices, top_scores = predict_next_token(model, tokenizer, test_sentence, device)

# Print results
print(f"Input: {test_sentence}")
print("Top 5 predicted next tokens:")
for i, (index, score) in enumerate(zip(top_indices, top_scores), 1):
    token = tokenizer.decode([index])
    print(f"{i}. '{token}' (score: {score.item():.4f})")

c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\torch\nn\modules\transformer.py:720: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  return torch._transformer_encoder_layer_fwd(


Input: The quick brown fox jumps over the lazy
Top 5 predicted next tokens:
1. ' looph' (score: 14.0095)
2. ' exha' (score: 12.0068)
3. '�士' (score: 11.4057)
4. ' tremend' (score: 9.3803)
5. ' millenn' (score: 9.2363)


In [34]:

# Method 2: Save only the state dictionary (recommended)
torch.save(custom_model.state_dict(), 'gpt2_starting_thoughtsformer.pth')